In [2]:
# library imports
import pandas as pd
import pyspark
import findspark
findspark.init()
from pyspark import SparkContext as sc
from pyspark.sql import SparkSession
from  pyspark.sql import functions as F

In [3]:
sc = sc('local', 'employee-survey')
spark = SparkSession(sc)

In [89]:
# import data
df = spark.read.csv('survey.csv', header=True)

In [90]:
# observe df
df.show()

+-------------------+---+------+--------------+-----+-------------+--------------+---------+--------------+--------------+-----------+------------+----------+------------+----------------+----------+----------+------------------+-------------------------+-----------------------+------------+------------+-----------------------+---------------------+------------------+---------------+--------------------+
|          Timestamp|Age|Gender|       Country|state|self_employed|family_history|treatment|work_interfere|  no_employees|remote_work|tech_company|  benefits|care_options|wellness_program| seek_help| anonymity|             leave|mental_health_consequence|phys_health_consequence|   coworkers|  supervisor|mental_health_interview|phys_health_interview|mental_vs_physical|obs_consequence|            comments|
+-------------------+---+------+--------------+-----+-------------+--------------+---------+--------------+--------------+-----------+------------+----------+------------+-------------

In [91]:
# drop NaN comments
df = df.filter(F.col('comments') != 'NA')

In [92]:
df = df.withColumn('Age', F.col('Age').cast('integer'))\
       .withColumn('Gender', F.lower(F.col('Gender')))\

df = df.withColumn('Gender', F.when(df.Gender == 'm', 'male')
                              .when(df.Gender == 'f', 'female')
                              .when(df.Gender == 'woman', 'female'))

df.show()

+-------+-------------------+-----------------+------+-------------+-----+-------------+--------------+---------+--------------+--------------+-----------+------------+----------+------------+----------------+----------+----------+----------+-------------------------+-----------------------+---------+----------+-----------------------+---------------------+------------------+---------------+--------------------+
|summary|          Timestamp|              Age|Gender|      Country|state|self_employed|family_history|treatment|work_interfere|  no_employees|remote_work|tech_company|  benefits|care_options|wellness_program| seek_help| anonymity|     leave|mental_health_consequence|phys_health_consequence|coworkers|supervisor|mental_health_interview|phys_health_interview|mental_vs_physical|obs_consequence|            comments|
+-------+-------------------+-----------------+------+-------------+-----+-------------+--------------+---------+--------------+--------------+-----------+------------+

In [129]:
df.select('comments').distinct().show()

+--------------------+
|            comments|
+--------------------+
|Some of these sho...|
|I'm diagnosed wit...|
|We don't fucking ...|
|Though it doesn't...|
|My company does p...|
|Many of these que...|
|Too many people e...|
|I burnt out this ...|
|I mostly suffer f...|
|When you are an i...|
|suffer from CR-PT...|
|Since being advis...|
|Thank you for you...|
|I have an excepti...|
|as a UK-based com...|
|My employer does ...|
|While I have not ...|
|Autism is a bitch...|
|I went through a ...|
|Hi Ed it's Paul D...|
+--------------------+
only showing top 20 rows



In [130]:
df = df.withColumn('user_id', F.monotonically_increasing_id())
df = df.select(*[
 'user_id',
 'Timestamp',
 'Age',
 'Gender',
 'Country',
 'state',
 'self_employed',
 'family_history',
 'treatment',
 'work_interfere',
 'no_employees',
 'remote_work',
 'tech_company',
 'benefits',
 'care_options',
 'wellness_program',
 'seek_help',
 'anonymity',
 'leave',
 'mental_health_consequence',
 'phys_health_consequence',
 'coworkers',
 'supervisor',
 'mental_health_interview',
 'phys_health_interview',
 'mental_vs_physical',
 'obs_consequence',
 'comments'
 ])

In [136]:
df.show()

+-------+-------------------+---+------+--------------+-----+-------------+--------------+---------+--------------+--------------+-----------+------------+----------+------------+----------------+----------+----------+------------------+-------------------------+-----------------------+------------+------------+-----------------------+---------------------+------------------+---------------+--------------------+
|user_id|          Timestamp|Age|Gender|       Country|state|self_employed|family_history|treatment|work_interfere|  no_employees|remote_work|tech_company|  benefits|care_options|wellness_program| seek_help| anonymity|             leave|mental_health_consequence|phys_health_consequence|   coworkers|  supervisor|mental_health_interview|phys_health_interview|mental_vs_physical|obs_consequence|            comments|
+-------+-------------------+---+------+--------------+-----+-------------+--------------+---------+--------------+--------------+-----------+------------+----------+--